数据集 CIFAR-10 python版

In [1]:
import _pickle as cPickle
import numpy as np
import os

In [2]:
CIFAR_DIR = './cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [3]:
with open(os.path.join(CIFAR_DIR, 'data_batch_1'), 'rb') as f:
    data = cPickle.load(f,encoding='iso-8859-1')
    print(type(data))
    print(data.keys())
    print(type(data['data']))
    print(type(data['labels']))
    print(type(data['batch_label']))
    print(data['data'].shape) # 10000个图片， 一张图片数据32 * 32 = 1024 * 3 = 3072 
    print(data['data'][0:2])  # RR--GG--BB 三个通道加在一起就是3072
    print(data['labels'][0:2])# 图片类型类型，数字表示
    print(data['batch_label'])#5个训练数据中的第一个 
    print(data['filenames'][0:2]) #图片名字
   

<class 'dict'>
dict_keys(['batch_label', 'labels', 'data', 'filenames'])
<class 'numpy.ndarray'>
<class 'list'>
<class 'str'>
(10000, 3072)
[[ 59  43  50 ... 140  84  72]
 [154 126 105 ... 139 142 144]]
[6, 9]
training batch 1 of 5
['leptodactylus_pentadactylus_s_000004.png', 'camion_s_000148.png']


In [4]:
image_arr = data['data'][100]
image_arr = image_arr.reshape((3,32,32)) # 32 32 3
image_arr = image_arr.transpose((1,2,0))
import matplotlib.pyplot as plt
#plt.imshow(image_arr)

构建模型

In [5]:
import tensorflow as tf

In [6]:
def load_data(filename):
    """ read data from data file"""
    with open(filename, 'rb') as f:
        data = cPickle.load(f,encoding='iso-8859-1')
        return data['data'], data['labels']

In [7]:
x = tf.placeholder(tf.float32, [None, 3072])
# [None], eg: [0,5,6,3]
y = tf.placeholder(tf.int64, [None])
# (3072 ,10)
w = tf.get_variable('w', [x.get_shape()[-1], 10], initializer = tf.random_normal_initializer(0,1))
# (10, )
b = tf.get_variable('b', [10], initializer=tf.constant_initializer(0.0))
# [None, 3072] * [3072, 10]  = [None, 10]
y_ = tf.matmul(x,w) + b

#多分类使用softmax
#通过Softmax回归，将logistic的预测二分类的概率的问题推广到了n分类的概率的问题
# e^(wx) / ∑k(e^wx)   k = 10
# mean sqare loss 平方差损失函数
#course: 1+e^x
#api: e^x / sum(e^x)
p_y = tf.nn.softmax(y_)
# 5 -> [0,0,0,0,0,1,0,0,0,0]
y_one_hot = tf.one_hot(y, 10, dtype = tf.float32)
loss = tf.reduce_mean(tf.square(y_one_hot - p_y))

#交叉熵损失函数
#loss = tf.losses.sparse_softmax_cross_entropy(labels=y,logits=y_)
# y_ 计算 sofmax    y 计算one_hot   loss 计算 ylog(y_)

#indeices
predict = tf.argmax(y_, 1)
# [1,0,1,1,1,0,0,0]
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))
# train是修改计算图中tf.Variable（默认是计算图中所有tf.Variable，可以通过var_list指定）达到训练的目的
with tf.name_scope('train_op'): #防止命名冲突,在该scope的命名的变量都是在train_op命名空间里的
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
#tensorflow.Dataset
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)  #十个类型的数据, 多分类
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 -1 # 每个数据之前是[0,255],缩放数据 [-1,1]之间，归一化,防止数据偏向一方
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
            pass
    
    def _shuffle_data(self):
        # [0,1,2,3,4,5] -> [5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size exmples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception('have no more examples.')
        if end_indicator > self._num_examples:
           raise Exception('batch size is lager than all examples.')
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
        pass
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d'%i)for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)
batch_data, batch_labels = train_data.next_batch(10)
# print(batch_data)
# print(batch_labels)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [9]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 100000
test_steps = 100
# 由于没有用到隐含层，故训练比较慢，准确率也比较低
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
        [loss, accuracy, train_op],
            feed_dict = {
                x:batch_data,
                y:batch_labels
            })
        if i % 500 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' \
                 % (i, loss_val, acc_val))
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data,
                        y: test_batch_labels
                    })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print ('[Test] Step: %d, acc %4.5f' %( i+1, test_acc))

[Train] Step: 0, loss: 0.17578, acc: 0.10000
[Train] Step: 500, loss: 0.16432, acc: 0.15000
[Train] Step: 1000, loss: 0.15137, acc: 0.20000
[Train] Step: 1500, loss: 0.17130, acc: 0.10000
[Train] Step: 2000, loss: 0.17508, acc: 0.05000
[Train] Step: 2500, loss: 0.13766, acc: 0.30000
[Train] Step: 3000, loss: 0.13258, acc: 0.30000
[Train] Step: 3500, loss: 0.14937, acc: 0.25000
[Train] Step: 4000, loss: 0.14797, acc: 0.20000
[Train] Step: 4500, loss: 0.08638, acc: 0.50000
(10000, 3072)
(10000,)
[Test] Step: 5000, acc 0.30900
[Train] Step: 5000, loss: 0.15817, acc: 0.20000
[Train] Step: 5500, loss: 0.15802, acc: 0.20000
[Train] Step: 6000, loss: 0.12585, acc: 0.35000
[Train] Step: 6500, loss: 0.08797, acc: 0.55000
[Train] Step: 7000, loss: 0.12949, acc: 0.35000
[Train] Step: 7500, loss: 0.13528, acc: 0.30000
[Train] Step: 8000, loss: 0.13427, acc: 0.30000
[Train] Step: 8500, loss: 0.14252, acc: 0.25000
[Train] Step: 9000, loss: 0.14665, acc: 0.25000
[Train] Step: 9500, loss: 0.15861, acc

[Train] Step: 75500, loss: 0.11779, acc: 0.40000
[Train] Step: 76000, loss: 0.10998, acc: 0.45000
[Train] Step: 76500, loss: 0.11596, acc: 0.40000
[Train] Step: 77000, loss: 0.13798, acc: 0.30000
[Train] Step: 77500, loss: 0.12601, acc: 0.35000
[Train] Step: 78000, loss: 0.12459, acc: 0.35000
[Train] Step: 78500, loss: 0.14396, acc: 0.25000
[Train] Step: 79000, loss: 0.15955, acc: 0.15000
[Train] Step: 79500, loss: 0.10110, acc: 0.45000
(10000, 3072)
(10000,)
[Test] Step: 80000, acc 0.38900
[Train] Step: 80000, loss: 0.10212, acc: 0.45000
[Train] Step: 80500, loss: 0.10595, acc: 0.45000
[Train] Step: 81000, loss: 0.11987, acc: 0.40000
[Train] Step: 81500, loss: 0.09479, acc: 0.50000
[Train] Step: 82000, loss: 0.10312, acc: 0.45000
[Train] Step: 82500, loss: 0.12761, acc: 0.35000
[Train] Step: 83000, loss: 0.11582, acc: 0.40000
[Train] Step: 83500, loss: 0.11879, acc: 0.40000
[Train] Step: 84000, loss: 0.13997, acc: 0.30000
[Train] Step: 84500, loss: 0.11893, acc: 0.40000
(10000, 3072)
